In [1]:
import boto3
import json
import aws
import cv2
import speech_recognition as sr
from playsound import playsound
import time

In [2]:
#Defining the variables
intro_output = 'intro.mp3'
see_output = "see.mp3"
res_output = "res.mp3"
bucketname = 'ssjnm'
photoname = 'test.jpg'
audio_output = 'out.mp3'
audio_input = 'input.mp3'
speechtext_output = 'textout.json'
record_output = 'record.wav'
jobname = 'Transcript'

In [3]:
#Camera shot function
def camerashot(photoname):
    cap = cv2.VideoCapture(0)
    status , photo = cap.read()
    cap.release()
    cv2.imwrite(photoname , photo)
    return photo

#S3
#Writing photo
def sendfiletos3(file_name, bucketname):
    s3 = boto3.resource('s3')
    try:
        s3.create_bucket( ACL='public-read',
          Bucket=bucketname,
          CreateBucketConfiguration={
              'LocationConstraint': 'ap-south-1'
          },)
        #print('Bucket Created')
    except :
        #print('Great ! This Bucket is already there')
        pass
    mybucket = s3.Bucket(bucketname)
    mybucket.put_object(  ACL='public-read',
      Body = open(file_name, 'rb'),
      Key=file_name,
      StorageClass='STANDARD_IA',
  )

#Rekognition
#Recognition the items for picture and function returning ssml for speaking
def rekophoto(photoname):
    reko = boto3.client('rekognition')
    detectinfo = reko.detect_labels( Image={
              'S3Object': {
                  'Bucket': 'ssjnm',
                  'Name': photoname,
              }
          },
    )
    see = "Well, If you ask me I can see "
    for i in range(len(detectinfo['Labels'])):
        items = detectinfo['Labels'][i]['Name']
        #print(items)
        see = see + items + " "
    return see

#Polly for speaking
#Using polly to speak up the passing Text and Audioname
def pollyspeak(text_input,audio_output):
    polly = boto3.client('polly')
    response = polly.synthesize_speech( Engine='standard',
        LanguageCode='en-US',
        OutputFormat='mp3', 
        Text=text_input,
        TextType='text',
          VoiceId='Justin',
      )
    with open(audio_output, 'wb') as filetowrite:
        filetowrite.write(response['AudioStream'].read())
    #file.close()

#Transcripting from speech to text and function returning speechtext
def transriptspeech(jobname,bucketname,audio_input,speechtext_output):
    
    transcript = boto3.client('transcribe')
    try:
        transcript.delete_transcription_job( TranscriptionJobName=jobname)
    except:
        pass
    transcript.start_transcription_job(TranscriptionJobName=jobname,
          LanguageCode='en-US',
          MediaFormat='wav',
          OutputBucketName=bucketname,
          Media={
              'MediaFileUri': 's3://' + bucketname + '/' + audio_input
          },
          OutputKey=speechtext_output,
      )
    status = transcript.get_transcription_job(TranscriptionJobName=jobname)
    while status['TranscriptionJob']['TranscriptionJobStatus'] != 'COMPLETED':
        time.sleep(20)
        status = transcript.get_transcription_job(TranscriptionJobName=jobname)
    s3 = boto3.resource('s3')
    mybucket = s3.Bucket(bucketname)
    mybucket.download_file(speechtext_output, speechtext_output)
    with open(speechtext_output) as f:
        data = json.load(f)
    out = data['results']['transcripts'][0]['transcript']
    return out

#Recording from microphone
def record(recordoutput):
    import sounddevice as sd
    from scipy.io.wavfile import write

    fs = 44100  # Sample rate
    seconds = 3  # Duration of recording

    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
    sd.wait()  # Wait until recording is finished
    write(recordoutput, fs, myrecording) 

In [4]:
intro = "Hello! I am SSJNM , What can I do for you ?"
print(intro)
pollyspeak(intro , intro_output)
playsound(intro_output)
print()
print('Listening.......')

# Listening and sending voice to S3
record(record_output)
sendfiletos3(record_output, bucketname)
text_out = transriptspeech(jobname,bucketname,record_output,speechtext_output)
print()
print("User: " + text_out)
print()

if ((("see" in text_out) or ("look" in text_out)) and ("camera" in text_out)) or (("see" in text_out) and ("can" in text_out)):
    res = "Okay! So you want me to look though your Camera"
    print(res)
    print()
    pollyspeak(res , res_output)
    playsound(res_output)
    
    #Capturing Image
    camerashot(photoname)
    #Sending photo to S3
    sendfiletos3(photoname, bucketname)
    #Recognizing what's there in the image
    see = rekophoto(photoname)
    #Answer
    print(see)
    pollyspeak(see , see_output)
    playsound(see_output)

Hello! I am SSJNM , What can I do for you ?

Listening.......

User: Can you tell me what you can see on camera?

Okay! So you want me to look though your Camera

Well, If you ask me I can see Wood Leisure Activities Hardwood Musical Instrument Flooring Person Human Plywood 


In [7]:
photo = cv2.imread(photoname)

In [8]:
cv2.imshow('My Pic', photo)